In [2]:
import numpy as np
import tensorflow as tf
import cv2
import requests
import os
from PIL import Image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
import efficientnet.tfkeras as efn
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [3]:
# Utilizando webscrapping en Unplash, descargamos una galería de imágenes de archivo de un 
# objeto determinado.

def descarga_imagen(imagen):
    # Access Key de Unsplash
    ACCESS_KEY = "0byE1o41WexZQJRM5Gy0N2NQIw36GLyZY6XcrUyf_M8"

    # Carpeta donde guardar las imágenes
    output_folder = f"./imagenes/{imagen}"
    os.makedirs(output_folder, exist_ok=True)

    # Palabra clave para buscar imágenes
    query = imagen  # Cambia esto por lo que necesites

    # Número total de imágenes deseadas
    total_images = 100  
    per_page = 30  # Máximo 30 imágenes por solicitud
    num_pages = (total_images // per_page) + 1  # Número de páginas necesarias

    # Descargar imágenes de varias páginas
    img_count = 0
    for page in range(1, num_pages + 1):  # Hacer solicitudes desde page=1 hasta page=num_pages
        print(f"🔍 Descargando página {page}...")

        # URL de la API con paginación
        url = f"https://api.unsplash.com/search/photos?query={query}&per_page={per_page}&page={page}&client_id={ACCESS_KEY}"

        # Hacer la solicitud a la API
        response = requests.get(url)

        # Intentar convertir la respuesta a JSON
        try:
            data = response.json()
        except Exception as e:
            print("Error al convertir la respuesta a JSON:", e)
            print("Respuesta de la API:", response.text)
            exit()

        # Verificar si "results" está en la respuesta
        if "results" not in data:
            print("ERROR: La API no devolvió 'results'. Esto es lo que devolvió:")
            print(data)  # Mostrar la respuesta completa para ver qué está pasando
            exit()

        # Si todo está bien, continuar con la descarga
        for img in data["results"]:
            if img_count >= total_images:
            print("Descarga completada.")
                break  # Detener si ya descargamos suficientes imágenes

            img_url = img["urls"]["full"]  # URL de la imagen en alta calidad
            img_response = requests.get(img_url)

            # Guardar la imagen
            img_path = os.path.join(output_folder, f"{query}_{img_count}.jpg")
            with open(img_path, "wb") as file:
                file.write(img_response.content)

            #print(f"Imagen descargada: {img_path}")
            img_count += 1
    return


IndentationError: expected an indented block after 'if' statement on line 47 (3024525705.py, line 48)

In [ ]:
#descarga_imagen("cheeseburger")

🔍 Descargando página 1...
🔍 Descargando página 2...
🔍 Descargando página 3...
🔍 Descargando página 4...


In [4]:
def predict_image(frame):
    # Cargar las etiquetas desde la URL proporcionada
    labels_url = 'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt'
    response = requests.get(labels_url)
    labels = response.text.splitlines()

    num_classes = len(labels)  # Aquí asumimos que hay tantas clases como etiquetas

    # Cargar el modelo base y añadir capas específicas de ajuste fino
    base_model = efn.EfficientNetB0(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation='relu')(x)
    predictions_layer = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=base_model.input, outputs=predictions_layer)

    # Congelar las capas base del modelo para entrenar solo las nuevas
    for layer in base_model.layers:
        layer.trainable = False

    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

    # Redimensionar y preprocesar el frame de entrada
    target_size = (224, 224)
    frame = Image.fromarray(np.uint8(frame)).resize(target_size)
    frame_array = np.array(frame) / 255.0
    frame_array = np.expand_dims(frame_array, axis=0)

    # Realizar la predicción
    prediction_probs = model.predict(frame_array)
    predicted_index = np.argmax(prediction_probs)
    predicted_label = labels[predicted_index]

    return predicted_label

In [6]:
# Con las imágenes descargadas, las pasamos por el modelo para ver cuáles identifica correctamente:

def predict_image(frame):
    model = tf.keras.applications.MobileNetV2(weights='imagenet')

    # Redimensionar y preprocesar la imagen
    img_resized = cv2.resize(frame, (224, 224))
    cv2.imshow('image',img_resized)
    cv2.waitKey(0)
    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_resized[np.newaxis, ...])
    
    # Realiza la predicción
    predictions = model.predict(img_array)
    predicted_index = np.argmax(predictions, axis=1)[0]
    
    # Cargar las etiquetas de ImageNet para poder interpretar el índice
    labels_path = tf.keras.utils.get_file('ImageNetLabels.txt',
                                          'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
    with open(labels_path, 'r') as f:
        labels = f.read().splitlines()
    
    predicted_label = labels[predicted_index]
    #print(f"La predicción del modelo es: {predicted_label}")
    return predicted_label



In [ ]:

# Pasamos la imagen por el modelo y guardamos en otra carpeta las que detecta correctamente:

def detecta_imagen(imagen):
    
    img_count = 0
    folder_path = f"./imagenes/{imagen}"
    #folder_path = os.path.join(".", "imagenes", imagen)  

    # Iterar sobre cada archivo en la carpeta
    for filename in os.listdir(folder_path):
        if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Filtra solo archivos de imagen
            file_path = os.path.join(folder_path, filename)  # Construye la ruta completa al archivo
            try:
                with Image.open(file_path) as img:  # Abre la imagen
                    img_resized = cv2.resize(img, (224, 224)) #Adaptamos la imagen al tamaño que lee bien el modelo.
                    img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_resized[np.newaxis, ...])
                    prediction = predict_image(img_array)
                    if prediction == imagen: # Guarda la imagen verificada en otra carpeta:
                        # Guardar la imagen
                        img_count += 1
                        output_folder = f"./imagenes/{imagen}_verificada"
                        img_path = os.path.join(output_folder, f"{imagen}_{img_count}.jpg")
                        with open(img_path, "wb") as file:
                            file.write(img_response.content)           
                    else:
                        print("No estoy seguro de qué estás haciendo. Intentemos otra vez.")
            except Exception as e:
                print(f"No se pudo procesar {filename}. Error: {e}")
    print(f"{img_count} imágenes procesadas correctamente")
    return

In [ ]:
img_count = 0
folder_path = f"./imagenes/cheeseburger"
#folder_path = os.path.join(".", "imagenes", imagen)  

# Iterar sobre cada archivo en la carpeta
for filename in os.listdir(folder_path):
    if filename.endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):  # Filtra solo archivos de imagen
        file_path = os.path.join(folder_path, filename)  # Construye la ruta completa al archivo
        img = cv2.imread(file_path)
        try:
            img_resized = cv2.resize(img, (224, 224)) #Adaptamos la imagen al tamaño que lee bien el modelo.
            img_array = tf.keras.applications.mobilenet_v2.preprocess_input(img_resized[np.newaxis, ...])
            prediction = predict_image(img_array)
            if prediction == "cheeseburger": # Guarda la imagen verificada en otra carpeta:
                # Guardar la imagen
                img_count += 1
                output_folder = f"./imagenes/cheeseburger_verificada"
                img_path = os.path.join(output_folder, f"cheeseburger_{img_count}.jpg")
                with open(img_path, "wb") as file:
                    file.write(img_response.content)           
            else:
                print("No estoy seguro de qué estás haciendo. Intentemos otra vez.")
        except Exception as e:
            print(f"No se pudo procesar {filename}. Error: {e}")
print(f"{img_count} imágenes procesadas correctamente")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 288ms/step
No estoy seguro de qué estás haciendo. Intentemos otra vez.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
No estoy seguro de qué estás haciendo. Intentemos otra vez.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 220ms/step
No estoy seguro de qué estás haciendo. Intentemos otra vez.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 214ms/step
No estoy seguro de qué estás haciendo. Intentemos otra vez.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 199ms/step
No estoy seguro de qué estás haciendo. Intentemos otra vez.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 239ms/step
No estoy seguro de qué estás haciendo. Intentemos otra vez.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 196ms/step
No estoy seguro de qué estás haciendo. Intentemos otra vez.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 242ms/step
No estoy seguro de qué estás haciendo. Intentemos otra vez.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 195ms/step
No estoy seguro de qué estás haciendo. Intentemos otra vez.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 174ms/step
No estoy seguro de qué estás haciendo. Intentemos otra vez.
1/1 ━━━━━━

In [8]:

image_path = os.path.join('.','imagenes/cheeseburger','cheeseburger_4.jpg')
frame = cv2.imread(image_path)

prediction = predict_image(frame)
print(prediction)



1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
bassoon


In [ ]:
detecta_imagen("cheeseburguer")

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '.\\imagenes\\cheeseburguer'